In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries neccesary for data preprocessing and vissualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix

In [ ]:
#due to space connstraints we will train our model on one parquet file lets take partition_id=4
train=pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet')

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
columns=list(train.columns)
columns

In [ ]:
def plotting_on_scattermatrix(s):
    scatterplotmatrix(train[s].values, figsize=(16, 14), names=s, alpha=0.5)
    plt.tight_layout()
    plt.show()
    

In [ ]:
#visualizing the first nine distribution to see how are some of the colums related ,fro this 
#we can see that feature 3 an 00 are some how lenearly depoenadent
cols=['date_id','time_id','symbol_id','weight','feature_00','feature_01','feature_02','feature_03','responder_6']
plotting_on_scattermatrix(cols)

In [ ]:
#from the below distribution we can see that most of the features are not much more correlated to each other
cols1=['feature_04','feature_05','feature_06','feature_07','feature_08','feature_09','feature_10','responder_6']
plotting_on_scattermatrix(cols1)

In [ ]:
#it clearly seems that the respondents are also clearly uncorrelated
cols2=['responder_1','responder_2','responder_3','responder_4','responder_5','responder_6','responder_7','responder_8']
plotting_on_scattermatrix(cols2)

In [ ]:
train.head()

In [ ]:
#lets check for the presence of null values 
missing_values = train.isnull().sum().reset_index()
missing_values.columns = ['Column', 'Missing Values']

# Display columns with missing values only
missing_values[missing_values['Missing Values'] > 0]

In [ ]:
print(missing_values)

In [ ]:
column_list=[]
for values in missing_values['Column']:
    column_list.append(values)
    
print(column_list)

In [ ]:
#the best possible way to deal with this null valued columns is to impoute them using mean due to our data distribution which 
#seem to be much more centered
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(train[column_list])
train[column_list] = imr.transform(train[column_list])

In [ ]:
missing_values = train.isnull().sum().reset_index()
missing_values.columns = ['Column', 'Missing Values']

# Display columns with missing values only
missing_values[missing_values['Missing Values'] > 0]

In [ ]:
train.head()

In [ ]:
X = train.drop(['responder_6'],axis=1)
y = train['responder_6']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
linear_reg = LinearRegression()
#svr_reg = SVR(kernel='sigmoid') # You can choose different kernels if needed
xgb_reg = XGBRegressor(random_state=42)
#rf_reg = RandomForestRegressor(random_state=42)

In [ ]:
slr = LinearRegression()
slr.fit(X_train, y_train)
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)


In [ ]:
 from sklearn.metrics import r2_score
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE train: %.3f, test: %.3f' % ( mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)))

In [ ]:
xgb_reg.fit(X_train, y_train)
y_train_pred = xgb_reg.predict(X_train)
y_test_pred = xgb_reg.predict(X_test)
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))
print('MSE train: %.3f, test: %.3f' % ( mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)))

In [ ]:
import kaggle_evaluation.jane_street_inference_server
def predict(test: pd.DataFrame, lags: pd.DataFrame | None = None) -> pd.DataFrame:
    """Make a prediction using XGBoost."""
    global lags_
    
    # Store the lags if provided
    if lags is not None:
        lags_ = lags

    # Use the model to make predictions
    predictions = xgb_reg.predict(test)

    # Create a DataFrame with predictions
    result = pd.DataFrame({
        'row_id': test['row_id'],
        'responder_6': predictions  # Assuming 'responder_6' is the target column
    })

    # Ensure the DataFrame structure is correct
    assert isinstance(result, pd.DataFrame)
    assert result.columns.tolist() == ['row_id', 'responder_6']
    assert len(result) == len(test)

    return result